In [3]:
import selenium
import pandas as pd
from selenium import webdriver

# Q1 & Q2 Web Scraping of product details searched in amazon.in

In [50]:
driver  = webdriver.Chrome("chromedriver.exe")

In [51]:
url = 'https://www.amazon.in/'
driver.get(url)

In [52]:
search_job = driver.find_element_by_id('twotabsearchtextbox')
search_job

<selenium.webdriver.remote.webelement.WebElement (session="88e4e1a302de4a2b2076de41f614b496", element="e9b61daf-7d69-431d-9fde-209375331df9")>

In [53]:
input = 'Guitar'
search_job.send_keys(input)

In [54]:
search_button = driver.find_element_by_id("nav-search-submit-button")
search_button.click()


In [55]:
urls = []
title_tags = driver.find_elements_by_xpath("//a[@class='a-link-normal s-no-outline']")
for i in title_tags:
    urls.append(i.get_attribute('href'))
urls


['https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A05211881ZGJIHX360ZMV&url=%2FKadence-Frontier-Acoustic-Guitar-Equalizer%2Fdp%2FB01HRIHJWY%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3DGuitar%26qid%3D1625403853%26sr%3D8-1-spons%26psc%3D1&qualifier=1625403853&id=2231746092166571&widgetName=sp_atf',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A04763212030HM2M4H8OM&url=%2FJUAREZ-Arp%25C3%25A9ggio-Acoustic-Guitar-Spruce%2Fdp%2FB08KGCFXKP%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3DGuitar%26qid%3D1625403853%26sr%3D8-2-spons%26psc%3D1&qualifier=1625403853&id=2231746092166571&widgetName=sp_atf',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A09285301NZJT1BBLJ0LW&url=%2FKadence-Frontier-Acoustic-Foldable-Strings%2Fdp%2FB01CJJGEVS%2Fref%3Dsr_1_3_sspa%3Fdchild%3D1%26keywords%3DGuitar%26qid%3D1625403853%26sr%3D8-3-spons%26psc%3D1&qualifier

In [56]:
# Getting links for 2nd and 3rd page
new_pages_links = []
new_pages = driver.find_elements_by_xpath("//li[@class='a-normal']//*[@href]")
for i in new_pages:
    new_pages_links.append(i.get_attribute('href'))
new_pages_links

['https://www.amazon.in/s?k=Guitar&page=2&qid=1625403853&ref=sr_pg_2',
 'https://www.amazon.in/s?k=Guitar&page=3&qid=1625403853&ref=sr_pg_3']

In [57]:
# Getting the website links of all the items in 2nd and 3rd pages
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
for i in new_pages_links:
    driver.get(i)
    delay =10
    WebDriverWait(driver,delay).until(EC.element_to_be_clickable((By.XPATH,"//li[@class='a-normal']//*[@href]")))
    title_tags = driver.find_elements_by_xpath("//a[@class='a-link-normal s-no-outline']")
    for i in title_tags:
        urls.append(i.get_attribute('href'))


In [58]:
len(urls)

162

In [137]:
from selenium.common.exceptions import StaleElementReferenceException

brand_name=[]
product_name = []
rating = []
no_of_ratings = []
price = []
return_or_exchange = []
delivery_by = []
availability = []
other_details = []
product_url = []

for i in urls:
    driver.get(i)
    product_url.append(i)
    #Scraping the brand names
    try:
        brand_tag = driver.find_element_by_xpath("//td[@class='a-span9']//span[@class='a-size-base']")
    except NoSuchElementException:
        pass
    try:
        if brand_tag.text is None:
            brand_name.append('-')
        else:
            brand_name.append(brand_tag.text)
    except StaleElementReferenceException:
        brand_name.append('-')
    #Scraping the Product names
    product_tag = driver.find_element_by_id("productTitle")
    if product_tag.text is None:
        product_name.append('-')
    else:
        product_name.append(product_tag.text)
        
    #Scraping the rating and No of Ratings
    try:
        rating_tag = driver.find_element_by_xpath("//td[@class='a-size-base']")
    except NoSuchElementException:
        pass
    try:
        if rating_tag.text is None:
            rating.append('-')
            no_of_ratings.append('-')
        else:
            rating.append(rating_tag.text.split('\n',1)[1])
            no_of_ratings.append(rating_tag.text.split('\n',1)[0])
    except StaleElementReferenceException:
            rating.append('-')
            no_of_ratings.append('-')
    #Scraping the Prices
    try:
        price_tag = driver.find_element_by_xpath("//td[@class='a-span12']")
    except NoSuchElementException:
        pass

    try:
        if price_tag.text is None:
            price.append('-')
            availability.append('Not Available')
        else:
            price.append(price_tag.text.replace('₹ ',''))
            availability.append('Available')
    except StaleElementReferenceException:
        price.append('-')
        availability.append('Not Available')

    

    #Scraping the Return or Exchange
    return_tag = driver.find_elements_by_xpath("//a[@class='a-size-small a-link-normal a-text-normal']")
    if len(return_tag) <3 :
        return_or_exchange.append(return_tag[0].text)
    else:
        return_or_exchange.append(return_tag[1].text)
        
    #Scraping the delivery by
    try:
        delivery_tag = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']//b")
    except NoSuchElementException:
        pass
    try:
        if delivery_tag.text is None:
            delivery_by.append('-')
        else:
            delivery_by.append(delivery_tag.text)
    except StaleElementReferenceException:
        delivery_by.append('-')  
         
        

In [138]:
product_info = pd.DataFrame()
product_info['Product'] = product_name
product_info['Brand'] = brand_name
product_info['Price'] = price
product_info['Rating'] = rating
product_info['No of Ratings'] = no_of_ratings
product_info['Return_or_exchange'] = return_or_exchange
product_info['Delivery_by'] = delivery_by
product_info['Availability'] = availability
product_info['Product_url'] = product_url

product_info

,Product,Brand,Price,Rating,No of Ratings,Return_or_exchange,Delivery_by,Availability,Product_url
0,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,Kadence,"6,499.00",3.9 out of 5 stars,868 ratings,7 Days Replacement,"Saturday, July 10",Available,https://www.amazon.in/gp/slredirect/picassoRed...
1,"JUAREZ Arpéggio 41 Inch Acoustic Guitar Kit, S...",JUAREZ,-,4.0 out of 5 stars,11 ratings,7 Days Replacement,-,Not Available,https://www.amazon.in/gp/slredirect/picassoRed...
2,"Kadence Frontier Series,Blue Acoustic Guitar W...",Kadence,"4,999.00",4.0 out of 5 stars,249 ratings,7 Days Replacement,"Saturday, July 10",Available,https://www.amazon.in/gp/slredirect/picassoRed...
3,Kadence Slowhand Premium Jumbo Semi Acoustic G...,Kadence,"10,499.00",4.2 out of 5 stars,223 ratings,7 Days Replacement,"Saturday, July 10",Available,https://www.amazon.in/gp/slredirect/picassoRed...
4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",Juârez,"2,499.00",3.9 out of 5 stars,"10,755 ratings",7 Days Replacement,"Saturday, July 10",Available,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...
157,FIRANDE Acoustic Guitar MF38C SunBurst 38 Inch...,FIRANDE,969.00,-,-,Amazon Delivered,"Friday, July 9",Available,https://www.amazon.in/FIRANDE-Acoustic-SunBurs...
158,JUAREZ Arpéggio 41 Inch Semi-Acoustic Guitar K...,JUAREZ,"6,579.00",4.6 out of 5 stars,5 ratings,7 Days Replacement,"Thursday, July 8",Available,https://www.amazon.in/JUAREZ-Arp%C3%A9ggio-Sem...
159,MUSTANG MT THM06 Guitar Pick (Pack of 6),MUSTANG,189.00,3.8 out of 5 stars,12 ratings,Amazon Delivered,"Tuesday, July 6",Available,https://www.amazon.in/MUSTANG-MT-THM06-Guitar-...
160,Pluto HW41C-201 41-inch Cutaway Acoustic Guita...,Pluto,"6,037.00",5.0 out of 5 stars,6 ratings,7 Days Replacement,"Saturday, July 17",Available,https://www.amazon.in/Pluto-HW41C201-HW41C-201...


In [139]:
driver.close()

# Q.3 Scraping Images from images.google.com

In [149]:
driver  = webdriver.Chrome("chromedriver.exe")

In [212]:
url = 'http://www.images.google.com/'
driver.get(url)

In [213]:
search_job = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_job

<selenium.webdriver.remote.webelement.WebElement (session="9a211e308a55276b5e4be0145d8f8da3", element="9b9865df-36b4-4251-96ee-0c4ed7a4e8ea")>

In [214]:
input = 'fruits'
search_job.send_keys(input)

In [215]:
search_button = driver.find_element_by_xpath("//button[@class='Tg7LZd']")
search_button.click()


In [216]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)

In [225]:
img_urls = []
img_tags = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")

for i in img_tags:
    i.click()
    actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
    for actual_image in actual_images:
        if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
            img_urls.append(actual_image.get_attribute('src'))    


In [234]:
os.chdir('D:\Fruits')
baseDir=os.getcwd()
for i, url in enumerate(img_urls[0:100]):
    file_name = f"{i:101}.jpg"
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(baseDir, file_name)
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

ERROR - COULD NOT SAVE https://7esl.com/wp-content/uploads/2017/12/fruit.jpg - cannot identify image file <_io.BytesIO object at 0x0000021479EBFA40>


In [240]:
from selenium.common.exceptions import ElementClickInterceptedException

url = 'http://www.images.google.com/'
driver.get(url)
search_job = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_job
input = 'Cars'
search_job.send_keys(input)
search_button = driver.find_element_by_xpath("//button[@class='Tg7LZd']")
search_button.click()
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)
img_urls = []
img_tags = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")

for i in range(0,len(img_tags)):
    img=img_tags[i]
    try:
        img.click()
        actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
        for actual_image in actual_images:
            if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                img_urls.append(actual_image.get_attribute('src'))  
    except ElementNotInteractableException or ElementClickInterceptedException as err:
        print(err)
os.chdir('D:\Cars')
baseDir=os.getcwd()
for i, url in enumerate(img_urls[0:100]):
    file_name = f"{i:101}.jpg"
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(baseDir, file_name)
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
    except Exception as e:
        pass

In [250]:
from selenium.common.exceptions import ElementNotInteractableException

url = 'http://www.images.google.com/'
driver.get(url)
search_job = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_job
input = 'Machine Learning'
search_job.send_keys(input)
search_button = driver.find_element_by_xpath("//button[@class='Tg7LZd']")
search_button.click()
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)
img_urls = []
img_tags = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")

for i in range(0,len(img_tags)):
    img=img_tags[i]
    try:
        img.click()
        actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
        for actual_image in actual_images:
            if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                img_urls.append(actual_image.get_attribute('src'))  
    except  (ElementClickInterceptedException, ElementNotInteractableException) as e:
        pass


os.chdir('D:\ML')
baseDir=os.getcwd()
for i, url in enumerate(img_urls[0:100]):
    file_name = f"{i:100}.jpg"
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(baseDir, file_name)
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
    except Exception as e:
        pass

In [5]:
driver.close()

# Q5. Scraping geospatial Coordinates

In [6]:
driver  = webdriver.Chrome("chromedriver.exe")

In [7]:
url = 'https://www.google.com/maps/'
driver.get(url)

In [8]:
search_job = driver.find_element_by_xpath("//input[@class='tactile-searchbox-input']")
search_job
input = 'Hyderabad'
search_job.send_keys(input)
search_button = driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']")
search_button.click()

In [11]:
current_url = driver.current_url

In [12]:
current_url

'https://www.google.com/maps/place/Hyderabad,+Telangana/@17.4121531,78.1278477,10z/data=!4m5!3m4!1s0x3bcb99daeaebd2c7:0xae93b78392bafbc2!8m2!3d17.385044!4d78.486671'

In [15]:
lat_long = current_url.split('@')
lat_long[1]

'17.4121531,78.1278477,10z/data=!4m5!3m4!1s0x3bcb99daeaebd2c7:0xae93b78392bafbc2!8m2!3d17.385044!4d78.486671'

In [18]:
latitude = lat_long[1].split(',')[0]
longitude = lat_long[1].split(',')[1]

In [17]:
latitude

'17.4121531'

In [19]:
longitude

'78.1278477'

In [20]:
driver.close()

# Q5. Scrape funding deals from 2nd quarter of 2020 from trak.in

In [21]:
driver  = webdriver.Chrome("chromedriver.exe")

In [22]:
url = 'https://www.trak.in/'
driver.get(url)

In [23]:
fund_tag = driver.find_element_by_xpath("//li[@class='menu-have-icon menu-icon-type-fontawesome menu-item menu-item-type-post_type menu-item-object-page better-anim-fade menu-item-51510']//a")
fund_url = fund_tag.get_attribute('href')

In [24]:
driver.get(fund_url)

In [41]:
from selenium.webdriver.support.select import Select

for i in (48,49,50):
    sel = Select(driver.find_element_by_xpath("//select[@name='tablepress-"+str(i)+"_length']"))
    sel.select_by_index(3)


In [82]:
rows = []
for i in (48,49,50):
    rws = (driver.find_elements_by_xpath("//table[@id='tablepress-"+str(i)+"']//tbody//tr"))
    print(len(rws))
    for j in rws:
        rows.append(j)
r = len(rows)

10
12
11


In [77]:
cols = driver.find_elements_by_xpath("//table[@id='tablepress-49']//tbody//tr[1]//td")
c = len(cols)


In [78]:
c

9

In [79]:
r

33

In [91]:
element = []
for k in (48,49,50):
    rws = (driver.find_elements_by_xpath("//table[@id='tablepress-"+str(k)+"']//tbody//tr"))
    r = len(rws)
    for i in range(1,r+1):
        row = []
        for j in range(1,c+1):
            d=driver.find_element_by_xpath("//table[@id='tablepress-"+str(k)+"']//tbody//tr["+str(i)+"]//td["+str(j)+"]").text
            row.append(d)
        element.append(row)


In [92]:
col_tags = driver.find_elements_by_xpath("//table[@id='tablepress-49']//thead//tr//th")
col_names = []
for i in col_tags:
    col_names.append(i.get_attribute('aria-label').replace(': activate to sort column ascending',''))
col_names

['Sr. No.',
 'Date (dd/mm/yyyy)',
 'Startup Name',
 'Industry / Vertical',
 'Sub-Vertical',
 'City / Location',
 "Investors' Name",
 'Investment Type',
 'Amount (In USD)']

In [94]:
df = pd.DataFrame(element)
df.head()

,0,1,2,3,4,5,6,7,8
0,1,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,2,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,3,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,4,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,5,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"


In [102]:
df.columns = col_names
df.head()

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,2,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,3,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,4,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,5,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
